In [129]:
import struct, inspect
import hexdump, zlib
import traceback

def decode_1(body):
    key = bytearray(b'\xf9\xa3\xff\xf8')
    ret = bytearray(body)
    ## 0F8FFA3F9h
    for i in range(len(body)):
        ret[i] = ret[i] ^ key[i%4]
    return bytes(ret)

def decode_2(body):
    compressed_len = struct.unpack('i', body[8:12])[0]
#     key = bytearray(b'\xf9\xa3\xff\xf8')
    key = struct.unpack('i', b'\xf9\xa3\xff\xf8')[0]
#     key = bytearray(struct.pack('i', 0xf8ffa3f9 - compressed_len))
    key = bytearray(struct.pack('i', key - compressed_len))
    ret = bytearray(body)
    ## 0F8FFA3F9h
    for i in range(len(body)):
        ret[i] = ret[i] ^ key[i%4]
    for i in (8, 9, 10, 11):
        ret[i] = ret[i] ^ key[i%4]
    return bytes(ret)

def decode_block_header(body):
    origin_len, compressed_len = struct.unpack('ii', body[8:16])
    key = struct.unpack('i', b'\xf9\xa3\xff\xf8')[0]
    compressed_len = compressed_len ^ key - origin_len
    key = struct.pack('i', key - origin_len)
    return key, compressed_len, origin_len

def decode_block_bak(body, key):
    ret = bytearray(body)
    key = bytearray(key)
#     end = ((len(body) - 4) >> 2) << 2  # last 4 bytes never encode
#     end_2 = (len(body) >> 4) << 4
    end = ((len(body) ) >> 2) << 2
    if end_2 > end:
        end = end_2
    for i in range(end):
        ret[i] = ret[i] ^ key[i%4]
    return ret

def decode_block_bak_2(body, key):
    ret = bytearray(body)
    key = bytearray(key)
    a2 = len(body) + 16
    v13 = a2 - 4;
    v15 = (a2 - 21) >> 2;
    v16 = v15 + 1;
    v17 = ((v15 - 3) >> 2) + 1;
    v18 = 4 * v17;
    v23 = 16 * v17 + 16;
    if v16 == v18:
        end = 16*v17
    else:
        end = 16*v17 + 4
        if v23 + 4 < v13:
            end += 4
            if v23 + 8 < v13:
                end += 4
    for i in range(end):
        ret[i] = ret[i] ^ key[i%4]
    return ret

def decode_block(body, key):
    ret = bytearray(body)
    key = bytearray(key)
    end = (len(body)-1) >> 2 << 2
    for i in range(end):
        ret[i] = ret[i] ^ key[i%4]
    return ret

level_shift_right = {19:9, 17:11, 15:12, 12:16, 10:16, 9:18, 7:20, 5:22}
level_shift_left = {19:7, 17:5, 15:4, 12:0, 10:0, 9:0, 7:0, 5:0}

def xy_2_idx(x, y, level):
    shift = level_shift_right[level]
    x = x >> shift
    y = y >> shift
    c3 = (x & 0x7) | ((y & 0x7) << 3)
    c2 = ((x>>3) & 0x7) | (((y>>3) & 0x7) << 3)
    c1 = ((x>>6) & 0x7) | (((y>>6) & 0x7) << 3)
    shift_c0 = level_shift_left[level]
    x = x >> 9
    y = y >> 9
    c0 = x | ( y << shift_c0 )
    return c0, c1, c2, c3

def idx_2_xy(idx, level):
    c0, c1, c2, c3 = idx
    t = 2**level_shift_left[level] - 1
    n = c0 & t
    n = (c1 & 0x7) | (n << 3)
    n = (c2 & 0x7) | (n << 3)
    n = (c3 & 0x7) | (n << 3)
    n = n << level_shift_right[level]
    x = n
    c0 = c0 >> level_shift_left[level]
    c1 = c1 >> 3
    c2 = c2 >> 3
    c3 = c3 >> 3
    n = c0 & t
    n = (c1 & 0x7) | (n << 3)
    n = (c2 & 0x7) | (n << 3)
    n = (c3 & 0x7) | (n << 3)
    n = n << level_shift_right[level]
    y = n
    return x, y

powers = {(1<<i):1 for i in range(17)}

def get_layer_by_id(block, id):
    for layer in block.layer:
        if layer.layerId == id:
            return layer

trick = [0]
        

class VMap():
    def __init__(self, filename='/tmp/shanghai_289.dat'):
        self.dz = zlib.decompressobj()
        content = open(filename, 'rb').read()
        self.content = content
        c = content
        self.size_all = len(c)
        s = struct.unpack('i', c[0x20:0x24])[0]
        self.size_h0 = 0x20 + 4 + s
        h1 = c[self.size_h0:self.size_h0+0x100]
        h1 = decode_1(h1)
        p = self.size_h0+0x100
        self.level_count = struct.unpack('i', h1[0x50:0x54])[0]
        self.size_trees = struct.unpack('i', h1[0x9C:0x9C+4])[0]
        levels = []
        self.levels = levels
        self.size_h1 = 0x100
        self.size_h2 = 0x40 * self.level_count
        h2 = c[p:p+self.level_count*0x40]
        h2 = decode_1(h2)
        self.levels_info = {}
        for i in range(self.level_count):
            levels.append(struct.unpack('h', h1[0x58+8*i:0x58+2+8*i])[0])
            root = struct.unpack('i', h2[0x40*i+0x10:0x40*i+0x10+4])[0]
            self.levels_info[levels[i]] = {}
            self.levels_info[levels[i]]['root'] = root
            s00, s01, s10, s11, s20, s21, s30, s31 = struct.unpack('BBBBBBBB', h2[0x40*i:0x40*i+8])
            assert s00 == s01 and s10 == s11 and s20 == s21 and s30 == s31
            self.levels_info[levels[i]]['idx_block_size'] = s00*s01, s10*s11, s20*s21, s30*s31 # idx item size not byte size
        for i in range(self.level_count-1):
            self.levels_info[levels[i]]['end'] = self.levels_info[levels[i+1]]['root']
        self.levels_info[levels[-1]]['end'] = self.size_h1+self.size_h2+self.size_trees
        h = self.size_h0 + self.size_h1 + self.size_h2
        self.idx_part = h1+h2+decode_1(c[h:h+self.size_trees-4])+c[h+self.size_trees-4:h+self.size_trees]
        
    def get_block_pos(self, loc):
#         header_offset = 0x2e ## 0x2f
        header = self.content[loc+self.size_h0:loc+self.size_h0+0x10]
        key, compressed_len, origin_len = decode_block_header(header)
        block = self.content[loc+self.size_h0+0x10:loc+self.size_h0+0x10+compressed_len]
        block = decode_block(block, key)
        dz = zlib.decompressobj()
        block = dz.decompress(block)[:origin_len]
        return block            

    def get_block_xy(self, xy, level):
        c0, c1, c2, c3 = xy_2_idx(xy[0], xy[1], level)
        n0 = self.levels_info[level]['root']
        p = n0 + c0*4
        n1 = struct.unpack('i', self.idx_part[p:p+4])[0]
        p = n1 + c1*4
        n2 = struct.unpack('i', self.idx_part[p:p+4])[0]
        p = n2 + c2*4
        n2 = struct.unpack('i', self.idx_part[p:p+4])[0]
        p = n2 + c3*4
        n3 = struct.unpack('i', self.idx_part[p:p+4])[0]
        header = self.content[n3+0x2f:n3+0x2f+0x10]
#         return header
        key, compressed_len, origin_len = decode_block_header(header)
        print(key, compressed_len, origin_len)
        block = self.content[n3+0x2f+0x10:n3+0x2f+0x10+compressed_len]
#         return block
        block = decode_block(block, key)
        assert block[:2] == b'\x78\x9C'
#         assert False
        dz = zlib.decompressobj()
        block = dz.decompress(block)[:origin_len]
        return block
        
    def __repr__(self):
        atts = {}
        for i in dir(self):
#             print(i)
            if i[0] == '_' or i in ('contentaa', ) :
                continue
            att = self.__getattribute__(i)
            if type(att) == bytes:
                continue
            if inspect.ismethod(att):
                continue
            atts[i] = att
#         print(type(atts))
#         print(atts.keys())
#         return 'gogogo'
        return '%s ==> %s'%('hello',atts.__repr__())
    
    def walk_idx_tree_bak(self, level):
        s0 = self.levels_info[level]['root']
        e0 = self.levels_info[level]['end']
        p0 = s0
        size_0 = 0
        size_1 = 0
        size_2 = 0
        size_3 = 0
        while True:
            v = struct.unpack('i', self.idx_part[p0:p0+4])[0]
            if v > 0:
                size_0 = v - s0
                assert size_0 in powers
                print('size_0', size_0)
                break
            p0 += 4
        e0 = s0 + size_0
        v0 = []
        for i in range(size_0>>2):
            v = struct.unpack('i', self.idx_part[s0+i*4:s0+i*4+4])[0]
            if v > 0: v0.append(i)
        if size_1 == 0 and len(v0) > 1:
            ii_0, ii_1 = s0 + v0[0]*4, s0 + v0[1]*4
            size_1 = struct.unpack('i', self.idx_part[ii_1:ii_1+4])[0] - struct.unpack('i', self.idx_part[ii_0:ii_0+4])[0]
            assert size_1 in powers
            print('size_1', size_1)
        for i_1 in v0:
            s1 = struct.unpack('i', self.idx_part[s0+i_1*4:s0+i_1*4+4])[0]
            p = s1
            if size_1 == 0:
                while True:
                    v = struct.unpack('i', self.idx_part[p:p+4])[0]
                    if v > 0:
                        size_1 = v - s1
                        assert size_1 in powers
                        print('size_1', size_1)
                        break
                    p += 4
            e1 = s1 + size_1
            v1 = []
            for i in range(size_1>>2):
                v = struct.unpack('i', self.idx_part[s1+i*4:s1+i*4+4])[0]
                if v > 0: v1.append(i)
            if size_2 ==0 and len(v1) > 1:
                ii_0, ii_1 = s1 + v1[0]*4, s1 + v1[1]*4
                size_2 = struct.unpack('i', self.idx_part[ii_1:ii_1+4])[0] - struct.unpack('i', self.idx_part[ii_0:ii_0+4])[0]
                assert size_2 in powers
                print('size_2', size_2)
            for i_2 in v1:
                s2 = struct.unpack('i', self.idx_part[s1+i_2*4:s1+i_2*4+4])[0]
                p = s2
                if size_2 == 0:
                    while True:
                        v = struct.unpack('i', self.idx_part[p:p+4])[0]
                        if v > 0:
                            size_2 = v - s2
                            assert size_2 in powers
                            print('size_2', size_2)
                            break
                        p += 4
                e2 = s2 + size_2
                v2 = []
                for i in range(size_2>>2):
                    v = struct.unpack('i', self.idx_part[s2+i*4:s2+i*4+4])[0]
                    if v > 0: v2.append(i)
                if size_3 == 0 and len(v2) > 1:
                    ii_0, ii_1 = s2 + v2[0]*4, s2 + v2[1]*4
                    size_3 = struct.unpack('i', self.idx_part[ii_1:ii_1+4])[0] - struct.unpack('i', self.idx_part[ii_0:ii_0+4])[0]
                    assert size_3 in powers
                    print('size_3', size_3)
                for i_3 in v2:
                    s3 = struct.unpack('i', self.idx_part[s2+i_3*4:s2+i_3*4+4])[0]
                    p = s3
                    if size_3 == 0:
                        while True:
                            v = struct.unpack('i', self.idx_part[p:p+4])[0]
                            if v > 0:
                                if v > self.levels_info[level]['end']:
                                    size_3 = self.levels_info[level]['end'] - s3
                                else:
                                    size_3 = v - s3
                                assert size_3 in powers
                                print('size_3', size_3)
                                break
                            p += 4
                    e3 = s3 + size_3
                    for i in range(size_3>>2):
                        v = struct.unpack('i', self.idx_part[s3+i*4:s3+i*4+4])[0]
                        if v > 0:
#                             print('###########> ', hex(i_1), i_1, i_2, i_3, i)
                            pass
                            if i_2 == 0x2b and i_3 == 0x16 and i == 0x3a:
                                print('###########> ', hex(i_1), i_1, i_2, i_3, i, hex(v))
            
    def walk_idx_tree(self, level, f = None):
        s0 = self.levels_info[level]['root']
        size_0, size_1, size_2, size_3 = self.levels_info[level]['idx_block_size']
        n = 0
        for i_0 in range(size_0):
            c0, = struct.unpack('i', self.idx_part[s0+i_0*4:s0+i_0*4+4])
            if c0 > 0:
                for i_1 in range(size_1):
                    c1, = struct.unpack('i', self.idx_part[c0+i_1*4:c0+i_1*4+4])
                    if c1 > 0:
                        for i_2 in range(size_2):
                            c2, = struct.unpack('i', self.idx_part[c1+i_2*4:c1+i_2*4+4])
                            if c2 >0:
                                for i_3 in range(size_3):
                                    c3, = struct.unpack('i', self.idx_part[c2+i_3*4:c2+i_3*4+4]); trick[0] = c3
                                    if c3 > 0:
                                        if n >= 500000:
                                            return
                                        else:
                                            pass
                                        try:
                                            block = self.get_block_pos(c3)
#                                             if len(block) != 136267:
#                                                 continue
                                            bl.Clear()
                                            bl.ParseFromString(block)
                                            if f:
                                                xy = idx_2_xy((i_0, i_1, i_2, i_3), 19)
                                                f(bl, xy)
                                            layer = get_layer_by_id(bl, 8)
                                            if layer:
#                                                 print('---------------------------', c3)
                                                n += 1
#                                             print('Oops', c3, i_0, i_1, i_2, i_3)
    #                                         print(get_layer_by_id(bl, 8))
#                                                 print(layer)
                                        except:
                                            n += 1
                                            traceback.print_exc()
                                            raise
                                            print('Oops', c3, i_0, i_1, i_2, i_3)
#                                         print('###########> ', hex(i_0), i_0, i_1, i_2, i_3)
                                        if i_1 == 0x2b and i_2 == 0x16 and i_3 == 0x3a:
                                            print('###########> ', hex(i_0), i_0, i_1, i_2, i_3)
            
import indoor_pb2
bl = indoor_pb2.Block()

def every_2_bits(raw):
    ret = []
    for n in raw:
        ret.append(n>>6)
        ret.append( (n>>4)&3 )
        ret.append( (n>>2)&3 )
        ret.append( n&3 )
    return ret

def decode_ints_with_bitmap(raw, bitmap):
    ret = []
    bitmap = every_2_bits(bitmap)
    n = 0
    for i in bitmap:
        item = 0
        for j in range(i+1):
            h = raw[n]
            item = item + (h<<(8*j))
            n += 1
        ret.append(item)
        if n == len(raw):
            break
    return ret

def lastbit_sign(a):
    if a&1:
        return (a>>1)*-1
    else:
        return a>>1
    
def pg(a, base_x=0, base_y=0, scale=1):
    x0, y0, a = a[0], a[1], a[2:]
    ret = [(x0,y0)]
    while a:
        dx, dy, a = a[0], a[1], a[2:]
        ret.append((ret[-1][0]+dx, ret[-1][1]+dy))
#     if ret[-1][0] - x0 < 0.1 and ret[-1][1] - y0 < 0.1:
#         ret[-1] = x0, y0
    ret = [(base_x+x/scale, base_y+y/scale) for (x, y) in ret]
    return ret

def pb_ints_2_points(ps, base_xy, scale=1):
    ps = [lastbit_sign(i) for i in ps]
    return pg(ps, base_xy[0], base_xy[1], scale)

import json

out_f = None

def f1(block, base_xy):
    x, y = base_xy
#     x, y = 0, 0
    layer = get_layer_by_id(bl, 8)
    if layer:
        for body in layer.body:
            for b2 in body.b2:
                val = decode_ints_with_bitmap(b2.bo.data, b2.bo.mask)
                val = [lastbit_sign(i) for i in val]
                val = pg(val, x, y, 10)
                if out_f:
#                     out_f.write('%d %d %d'% (trick[0], base_xy[0], base_xy[1]))
                    out_f.write(json.dumps({'outline':val, 'height':b2.bo.height}))
                    out_f.write('\n')
                else:
                    print(json.dumps(val))
            

def f_poi(block, base_xy):
    x, y = base_xy
    layer = get_layer_by_id(bl, 3)
    if layer:
        for body in layer.body:
            for b2 in body.b2:
                if out_f:
                    out_f.write(json.dumps({'tag':body.tag, 'poi':b2.poi.poi, 'location':(x+b2.poi.x/100, y+b2.poi.y/100)}, ensure_ascii=False))
                    out_f.write('\n')

In [151]:
def combine_indoor_info(infos):
    ret = {}
    ret['name'] = infos[0]['name']
    ret['floor_names'] = infos[0]['floor_names']
    ret['border'] = None
    ret['floors'] = []
    tmp_floors = {}
    floor_names = infos[0]
    for info in infos:
        assert ret['floor_names'] == info['floor_names']
        if ret['border']:
            assert ret['border'] == info['border']
        else:
            ret['border'] = info['border']
        for floor in info['floors']:
            if floor['name'] not in tmp_floors:
                floor_info = {}
                floor_info['name'] = floor['name']
                floor_info['pois'] = [] + floor['pois']
                floor_info['polygons'] = [] + floor['polygons']
                tmp_floors[floor['name']] = floor_info
            else:
                floor_info = tmp_floors[floor['name']]
                assert floor_info['name'] == tmp_floors[floor['name']]['name']
                floor_info['pois'] = floor['pois'] + tmp_floors[floor['name']]['pois']
                floor_info['polygons'] = floor['polygons'] + tmp_floors[floor['name']]['polygons']
    for floor_name in ret['floor_names']:
        ret['floors'].append(tmp_floors[floor_name])
    return ret

In [136]:
g_tmp = {}

bbb = []

def f_indoor(block, base_xy):
    for indoor in block.indoor:
        if indoor.meta.some_id == '5087719067573934800':
            print('-------------------------------------------------')
            bbb.append(block)
            print(indoor)
        indoor_info = {}
        indoor_info['name'] = indoor.meta.name
        indoor_info['floor_names'] = list(indoor.meta.floors)
        if indoor.meta.border.points:
            indoor_info['border'] = pb_ints_2_points(indoor.meta.border.points, base_xy, 200)
        else:
            indoor_info['border'] = []
        indoor_info['floors'] = []
        for floor in indoor.floor:
            floor_info = {}
            floor_info['name'] = floor.name
            floor_info['pois'] = []
            floor_info['polygons'] = []
            for b1 in floor.b1:
                for b2 in b1.b2:
                    for b3 in b2.b3:
                        for b4 in b3.b4:
                            for poi in b4.poi:
                                poi_info = {}
                                p = []
                                poi_info['loc'] = (base_xy[0]+poi.x/200, base_xy[1]+poi.y/200)
                                poi_info['tag_3'] = poi.attr_3
                                poi_info['name'] = poi.name
                                floor_info['pois'].append(poi_info)
                            for polygon in b4.b50:
                                polygon_info = {}
#                                 polygon_info['points'] = list(polygon.points)
#                                 ps = [lastbit_sign(i) for i in polygon.points]
                                polygon_info['points'] = pb_ints_2_points(polygon.points, base_xy, 200)
                                polygon_info['tag'] = b3.some_id_b3
                                floor_info['polygons'].append(polygon_info)
            indoor_info['floors'].append(floor_info)
        if indoor.meta.some_id not in g_tmp:
            g_tmp[indoor.meta.some_id] = []
        g_tmp[indoor.meta.some_id].append(indoor_info)
        if out_f:
            out_f.write(json.dumps(indoor_info, ensure_ascii=False))
            out_f.write('\n')

In [39]:
list(range(10))/100

TypeError: unsupported operand type(s) for /: 'list' and 'int'

In [16]:
xy_2_idx(12640512, 2519296, 19)

(1200, 33, 62, 0)

In [135]:
# m1 = VMap('/tmp/aomentebiexingzheng_2911.dat')
# p = 80085
# cc = m1.content[p:p+100]
# hexdump.hexdump(cc)
# decode_block_header(cc[0x2e:0x2e+0x10])

In [55]:
m1

hello ==> {'dz': <zlib.Decompress object at 0x7f8a80bdeca8>, 'level_count': 4, 'levels': [19, 17, 15, 12], 'levels_info': {19: {'root': 512, 'idx_block_size': (16384, 64, 64, 64), 'end': 103424}, 17: {'root': 103424, 'idx_block_size': (1024, 64, 64, 64), 'end': 112896}, 15: {'root': 112896, 'idx_block_size': (256, 64, 64, 64), 'end': 115968}, 12: {'root': 115968, 'idx_block_size': (1, 64, 64, 64), 'end': 116740}}, 'size_all': 1996187, 'size_h0': 46, 'size_h1': 256, 'size_h2': 256, 'size_trees': 116228}

In [57]:
m2

hello ==> {'dz': <zlib.Decompress object at 0x7f8a80bde810>, 'level_count': 4, 'levels': [19, 17, 15, 12], 'levels_info': {19: {'root': 512, 'idx_block_size': (16384, 64, 64, 64), 'end': 287232}, 17: {'root': 287232, 'idx_block_size': (1024, 64, 64, 64), 'end': 309504}, 15: {'root': 309504, 'idx_block_size': (256, 64, 64, 64), 'end': 316928}, 12: {'root': 316928, 'idx_block_size': (1, 64, 64, 64), 'end': 317956}}, 'size_all': 146255037, 'size_h0': 47, 'size_h1': 256, 'size_h2': 256, 'size_trees': 317444}

In [133]:
# m2 = VMap('/tmp/shanghai_289.dat')
# p = 0
# cc = m2.content[p:p+0x100]
# hexdump.hexdump(cc)

In [132]:
# m1 = VMap('/tmp/jiayuguan_33.dat')
# p = 0
# cc = m1.content[p:p+0x100]
# hexdump.hexdump(cc)

In [131]:
# p = 0
# cc = m1.content[p:p+0x100]
# hexdump.hexdump(cc)

In [130]:
# p = 0
# cc = m1.content[p:p+0x3000]
# hexdump.hexdump(m1.idx_part)

In [134]:
# m1 = VMap('/tmp/shanghai_289.dat')
# p = 412578
# cc = m1.content[p:p+100]
# hexdump.hexdump(cc)
# decode_block_header(cc[0x2f:0x2f+0x10])

In [17]:
bl = m1.get_block_xy((12640512, 2519296), 19)

b'?\xa3\xff\xbc' -1928897114 1006633146


AssertionError: 

In [12]:
m1

hello ==> {'dz': <zlib.Decompress object at 0x7f8a80c4b810>, 'level_count': 4, 'levels': [19, 17, 15, 12], 'levels_info': {19: {'root': 512, 'idx_block_size': (16384, 64, 64, 64), 'end': 69120}, 17: {'root': 69120, 'idx_block_size': (1024, 64, 64, 64), 'end': 73984}, 15: {'root': 73984, 'idx_block_size': (256, 64, 64, 64), 'end': 75776}, 12: {'root': 75776, 'idx_block_size': (1, 64, 64, 64), 'end': 76548}}, 'size_all': 1319774, 'size_h0': 46, 'size_h1': 256, 'size_h2': 256, 'size_trees': 76036}

In [29]:
g_tmp['1264953905456873471'][0]

KeyError: '1264953905456873471'

In [40]:
len(g_tmp)

500

In [42]:
[(len(g_tmp[i]), i) for i in g_tmp if len(g_tmp[i])>1]

[(2, '7291462937612809574'),
 (2, '13269978721838070726'),
 (2, '1264953364290994175'),
 (2, '1264928432005840895'),
 (4, '13141295276338611181'),
 (2, '18237050984920854222'),
 (2, '14521289630102608789'),
 (4, '11399872943630822777'),
 (2, '4261905614072172325'),
 (2, '3643429619836701675'),
 (2, '15526309407321075645'),
 (2, '4890931751490234016'),
 (4, '17998070834823954431'),
 (4, '1264952071505838079'),
 (2, '4051545313113984409'),
 (3, '1261266414040252415'),
 (4, '18234824920217506221'),
 (2, '5943969799765213036'),
 (2, '17351440145062994561'),
 (4, '17625930756352114687'),
 (4, '1261327677453762559'),
 (2, '1338149182037462045'),
 (3, '1264900738056716287'),
 (2, '1264900596322795519'),
 (2, '1006927620767235763'),
 (2, '1264869711212969983'),
 (2, '17838998927472281616'),
 (2, '6250959102217293761'),
 (2, '17998204137723920383'),
 (2, '12874811800780117735'),
 (4, '3619783904913940359'),
 (2, '6306883150453921025'),
 (2, '7587493299578957247'),
 (2, '8614500397080393884'),
 

In [61]:
g_tmp['5087719067573934800'][0]['floor_names']

['B2', 'B1', 'M', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6']

In [152]:
combine_indoor_info(g_tmp['5087719067573934800'])

{'border': [(13505602.55, 3636384.81),
  (13505582.65, 3636384.05),
  (13505568.19, 3636383.45),
  (13505559.76, 3636383.08),
  (13505527.73, 3636381.91),
  (13505524.28, 3636381.5),
  (13505521.83, 3636380.32),
  (13505519.61, 3636378.23),
  (13505518.61, 3636375.89),
  (13505518.27, 3636373.3),
  (13505518.27, 3636370.71),
  (13505518.49, 3636365.92),
  (13505519.03, 3636350.25),
  (13505519.79, 3636329.53),
  (13505532.3, 3636329.73),
  (13505535.8, 3636329.76),
  (13505544.58, 3636329.94),
  (13505545.1, 3636313.87),
  (13505542.9, 3636313.74),
  (13505520.33, 3636312.69),
  (13505520.87, 3636295.99),
  (13505521.42, 3636281.48),
  (13505521.96, 3636267.11),
  (13505522.51, 3636251.44),
  (13505522.81, 3636244.34),
  (13505522.83, 3636243.93),
  (13505522.94, 3636241.47),
  (13505533.73, 3636241.8),
  (13505535.03, 3636205.28),
  (13505560.49, 3636206.21),
  (13505561.91, 3636206.26),
  (13505571.16, 3636206.53),
  (13505589.06, 3636207.15),
  (13505608.19, 3636207.91),
  (13505610

In [76]:
# bbb[1]

In [137]:
# m1 = VMap('/tmp/aomentebiexingzheng_2911.dat')
m1 = VMap('/tmp/shanghai_289.dat')
# m1 = VMap('/tmp/ss.dat')
# m1

out_f = open('/tmp/shanghai_289_indoor.json', 'w')

%time m1.walk_idx_tree(19, f_indoor)

out_f.close()
# m1 = VMap('/tmp/quanguogailue.dat')
# m1.walk_idx_tree(10)


###########>  0x6b3 1715 43 22 58
-------------------------------------------------
floor {
  name: "B2"
  idx_num: -3
  b1 {
    b2 {
      some_id: 7
      b3 {
        some_id_b3: 20642
        b4 {
          b50 {
            attr_1: 248
            points: 204800
            points: 204800
            points: 2777
            points: 1
            points: 80
            points: 2633
            points: 88
            points: 3057
            points: 2608
            points: 56
            points: 1
            points: 5632
            attr_5: 1
          }
        }
      }
      b3 {
        some_id_b3: 4
        b4 {
          b50 {
            attr_1: 248
            points: 204800
            points: 202228
            points: 2697
            points: 61
            points: 88
            points: 3057
            points: 2608
            points: 56
            points: 1
            points: 3060
          }
        }
      }
      b3 {
        some_id_b3: 10
        b4 {
      

-------------------------------------------------
floor {
  name: "B2"
  idx_num: -3
  b1 {
    b2 {
      some_id: 3
      b3 {
        some_id_b3: 18
        b4 {
          poi {
            x: 7613
            y: 4662
            attr_3: 192
            attr_4: 5087719067573934800
            attr_5: 20000
            name: "\350\231\271\346\241\245\345\244\251\345\234\260\\\350\264\255\347\211\251\344\270\255\345\277\203"
            attr_10: 1
          }
        }
      }
      b3 {
        some_id_b3: 57
        b4 {
          poi {
            x: 12202
            y: 269
            attr_3: 252
            attr_4: 4371708599591697449
            attr_5: 19500
            name: "\345\201\234\350\275\246\345\234\272\345\207\272\345\205\245\345\217\243"
            attr_10: 1
          }
        }
      }
      b3 {
        some_id_b3: 158
        b4 {
          poi {
            x: 5562
            y: 11995
            attr_3: 60
            attr_4: 6194376336043092866
          

CPU times: user 1min, sys: 1.1 s, total: 1min 2s
Wall time: 1min 2s


In [5]:
%debug

> <ipython-input-2-e8185311019d>(380)pg()
    378 
    379 def pg(a, base_x=0, base_y=0, scale=1):
--> 380     x0, y0, a = a[0], a[1], a[2:]
    381     ret = [(x0,y0)]
    382     while a:

ipdb> a
a = []
base_x = 13508096
base_y = 3579392
scale = 200
ipdb> ps
*** NameError: name 'ps' is not defined
ipdb> quit


In [389]:
def decode_b(body):
    a2 = body + 16
    v13 = a2 - 4;
    v15 = (a2 - 21) >> 2;
    v16 = v15 + 1;
    v17 = ((v15 - 3) >> 2) + 1;
    v18 = 4 * v17;
    v23 = 16 * v17 + 16;
    if v16 == v18:
        end = 16*v17
    else:
        end = 16*v17 + 4
        if v23 + 4 < v13:
            end += 4
            if v23 + 8 < v13:
                end += 4
    return end

def decode_b2(body):
    end = body >> 2 << 2
    if end == body:
        end -= 4
    print( ((body -1) >> 2 << 2) - end )
    return end

for i in range(16, 32):
    print(decode_b(i), i>>2 <<2, i)

12 16 16
16 16 17
16 16 18
16 16 19
16 20 20
20 20 21
20 20 22
20 20 23
20 24 24
24 24 25
24 24 26
24 24 27
24 28 28
28 28 29
28 28 30
28 28 31


In [347]:
idx_2_xy((1715, 36, 62, 18), 19)

(13526016, 3568640)

In [9]:
m1 = VMap()
# b = m1.get_block_xy((13497344, 3618304), 19)
b = m1.get_block_xy((13497344, 3618304), 19)
len(b)
bl.Clear()
bl.ParseFromString(b)
# bl

b'y\x89\xfd\xf8' 66174 137856


137856

In [62]:
open('/tmp/indoor.bin', 'wb').write(b)

137856

In [85]:
hexdump.hexdump(b)

00000000: 0A 86 03 08 07 12 26 08  C4 0C 12 21 12 1F 08 F8  ......&....!....
00000010: 01 22 01 01 32 0F E0 03  CA 03 A9 09 0A 17 01 80  ."..2...........
00000020: 02 26 26 09 F6 3A 04 50  10 00 00 50 11 12 83 01  .&&..:.P...P....
00000030: 08 8D 0C 12 29 12 27 08  F8 01 22 01 01 32 16 94  ....).'..."..2..
00000040: 7A 02 3D 23 12 9F 01 1A  2B D0 01 DE 30 15 12 01  z.=#....+...0...
00000050: 1F 6A 6D 01 3E 3A 05 10  10 00 10 40 50 11 12 2E  .jm.>:.....@P...
00000060: 12 2C 08 F8 01 22 01 01  32 12 3C 03 AE 01 01 03  .,..."..2.<.....
00000070: 0E A1 76 0C 0A 0A 01 62  0F 14 81 16 3A 05 50 00  ..v....b....:.P.
00000080: 00 00 00 42 07 08 02 10  F8 01 18 00 50 12 12 23  ...B........P..#
00000090: 12 21 08 F8 01 22 01 01  32 11 26 02 42 02 22 83  .!..."..2.&.B.".
000000A0: 01 B0 01 08 01 68 15 B6  BF 01 5C 3A 04 51 40 04  .....h....\:.Q@.
000000B0: 00 50 11 12 3E 08 8C DE  03 12 38 12 36 08 F8 01  .P..>.....8.6...
000000C0: 22 01 01 32 22 F2 01 00  04 95 01 01 02 D5 08 3D  "..2"..........=

00004820: 32 38 50 01 12 22 2A 20  08 BA DC 02 10 8E D4 02  28P.."* ........
00004830: 18 3C 20 A4 EF ED A8 9B  93 CE 96 7D 28 EE 82 01  .< ........}(...
00004840: 42 04 4D 30 32 39 50 01  12 22 2A 20 08 B6 DF 02  B.M029P.."* ....
00004850: 10 95 D4 02 18 3C 20 95  B1 BE A8 D5 EA AD B2 10  .....< .........
00004860: 28 EE 82 01 42 04 4D 30  33 30 50 01 12 22 2A 20  (...B.M030P.."* 
00004870: 08 AD E5 02 10 D0 CF 02  18 3C 20 C6 85 D1 E4 AA  .........< .....
00004880: 98 8B E2 77 28 EE 82 01  42 04 4D 30 33 39 50 01  ...w(...B.M039P.
00004890: 12 22 2A 20 08 9E E8 02  10 D7 CF 02 18 3C 20 80  ."* .........< .
000048A0: EE C2 A4 F5 C8 9D BC 76  28 EE 82 01 42 04 4D 30  .......v(...B.M0
000048B0: 33 38 50 01 12 22 2A 20  08 9A EB 02 10 DE CF 02  38P.."* ........
000048C0: 18 3C 20 EC EC 86 A6 F4  C1 91 91 3D 28 EE 82 01  .< ........=(...
000048D0: 42 04 4D 30 33 37 50 01  12 23 2A 21 08 A7 E2 02  B.M037P..#*!....
000048E0: 10 9C D4 02 18 3C 20 C3  CA C7 94 E3 CB EA AE 81  .....< .........

00009670: 99 76 28 BE 79 42 06 E6  A5 BC E6 A2 AF 50 01 12  .v(.yB.......P..
00009680: 24 2A 22 08 8F A7 02 10  F2 A6 02 18 3C 20 85 95  $*".........< ..
00009690: 88 D2 ED F2 DB 89 B7 01  28 BE 79 42 06 E6 A5 BC  ........(.yB....
000096A0: E6 A2 AF 50 01 12 24 2A  22 08 E9 A9 02 10 AC A6  ...P..$*".......
000096B0: 02 18 3C 20 E3 81 C2 81  C2 A2 FD 89 9F 01 28 BE  ..< ..........(.
000096C0: 79 42 06 E6 A5 BC E6 A2  AF 50 01 12 BC 01 08 35  yB.......P.....5
000096D0: 12 23 2A 21 08 F1 62 10  A3 D5 02 18 3C 20 D8 FD  .#*!..b.....< ..
000096E0: C2 DA 9C BD FB DC 87 01  28 BE 79 42 06 E6 89 B6  ........(.yB....
000096F0: E6 A2 AF 50 01 12 23 2A  21 08 FB 60 10 DB D5 02  ...P..#*!..`....
00009700: 18 3C 20 CE D9 A2 BA 99  B0 E4 E9 FE 01 28 BE 79  .< ..........(.y
00009710: 42 06 E6 89 B6 E6 A2 AF  50 01 12 24 2A 22 08 98  B.......P..$*"..
00009720: A5 02 10 9E 9E 02 18 3C  20 D2 F7 E5 AF 90 8B D2  .......< .......
00009730: C7 FD 01 28 BE 79 42 06  E6 89 B6 E6 A2 AF 50 01  ...(.yB.......P.

0000DF20: 0B E8 01 12 1D 12 1B 08  F8 01 12 16 BC A2 09 98  ................
0000DF30: A6 0B 91 03 AD 13 94 0B  99 02 94 03 AC 13 99 0B  ................
0000DF40: 98 02 12 1D 12 1B 08 F8  01 12 16 D4 AD 09 80 A4  ................
0000DF50: 0B 95 03 AD 13 98 0B E9  01 90 03 AC 13 95 0B E8  ................
0000DF60: 01 12 1D 12 1B 08 F8 01  12 16 E8 B8 09 98 A2 0B  ................
0000DF70: 91 03 AD 13 94 0B E5 01  94 03 AC 13 99 0B E4 01  ................
0000DF80: 12 19 12 17 08 F8 01 12  12 C4 DE 09 A0 A7 0B B9  ................
0000DF90: 09 21 58 D1 12 B8 09 24  59 CC 12 12 19 12 17 08  .!X....$Y.......
0000DFA0: F8 01 12 12 8C D5 09 80  A7 0B E5 09 21 5C CD 12  ............!\..
0000DFB0: E0 09 1C 59 D0 12 12 1D  12 1B 08 F8 01 12 16 80  ...Y............
0000DFC0: 89 09 D4 AA 0B 95 03 AD  13 98 0B 9D 02 90 03 E0  ................
0000DFD0: 13 95 0B E8 01 12 20 12  1E 08 F8 01 12 19 94 88  ...... .........
0000DFE0: 08 E8 8A 0B E1 01 89 0A  CC 14 CD 03 88 01 EC 06  ................

00013CD0: 09 B1 01 B4 01 D8 09 89  09 B0 01 12 1C 12 1A 08  ................
00013CE0: F8 01 12 15 A0 8C 03 A0  E5 0A D5 06 D9 28 A4 06  .............(..
00013CF0: AD 01 D4 06 90 29 A5 06  74 12 1D 12 1B 08 F8 01  .....)..t.......
00013D00: 12 16 A4 B2 03 D4 F8 09  E1 01 D5 09 E4 09 E5 01  ................
00013D10: E0 01 D4 09 E5 09 E4 01  12 1D 12 1B 08 F8 01 12  ................
00013D20: 16 84 B4 03 AC 82 0A E1  01 D9 09 E4 09 E5 01 DC  ................
00013D30: 01 D4 09 E1 09 E8 01 12  19 12 17 08 F8 01 12 12  ................
00013D40: B8 E6 03 80 C0 0C D1 06  01 54 E9 28 D0 06 24 55  .........T.(..$U
00013D50: C4 28 12 19 12 17 08 F8  01 12 12 DC D4 09 A4 E0  .(..............
00013D60: 0A 01 91 0C 98 0B 19 01  90 0C 99 0B 18 12 1B 12  ................
00013D70: 19 08 F8 01 12 14 DC DB  09 B8 8F 0B 81 0F 49 B0  ..............I.
00013D80: 01 D5 16 84 0F 48 B5 01  D4 16 12 19 12 17 08 F8  .....H..........
00013D90: 01 12 12 C8 C9 09 B8 E0  0A 01 8D 0C 94 0B 19 01  ................

00019A90: A1 0C A0 02 12 1D 12 1B  08 F8 01 12 16 A8 B3 03  ................
00019AA0: EC BC 09 E1 01 F1 0A 9C  0C A1 02 E0 01 F0 0A 9D  ................
00019AB0: 0C A0 02 12 1D 12 1B 08  F8 01 12 16 AC 92 09 9C  ................
00019AC0: A4 09 F1 03 99 16 D0 0D  9D 02 EC 03 94 16 CD 0D  ................
00019AD0: A0 02 12 1D 12 1B 08 F8  01 12 16 A4 B2 03 D4 F8  ................
00019AE0: 09 E1 01 D5 09 E4 09 E5  01 E0 01 D4 09 E5 09 E4  ................
00019AF0: 01 12 1D 12 1B 08 F8 01  12 16 84 B4 03 AC 82 0A  ................
00019B00: E1 01 D9 09 E4 09 E5 01  DC 01 D4 09 E1 09 E8 01  ................
00019B10: 12 1D 12 1B 08 F8 01 12  16 E4 B5 03 80 8C 0A E1  ................
00019B20: 01 D5 09 E0 09 E9 01 E0  01 D8 09 E1 09 E4 01 12  ................
00019B30: 1D 12 1B 08 F8 01 12 16  CC FA 05 E0 A4 0B B5 01  ................
00019B40: 85 06 C0 2F A9 0B B4 01  84 06 C1 2F A8 0B 12 1D  .../......./....
00019B50: 12 1B 08 F8 01 12 16 98  F9 05 DC 9E 0B B1 01 85  ................

0001C100: 02 18 3C 20 FB F0 8C CD  D5 BD E2 84 71 28 BE 79  ..< ........q(.y
0001C110: 42 06 E6 A5 BC E6 A2 AF  50 01 12 23 2A 21 08 F1  B.......P..#*!..
0001C120: F1 01 10 B8 CC 02 18 3C  20 8A F4 E1 C4 8E E4 B3  .......< .......
0001C130: EB 66 28 BE 79 42 06 E6  A5 BC E6 A2 AF 50 01 12  .f(.yB.......P..
0001C140: 23 2A 21 08 E7 A5 02 10  9A A6 02 18 3C 20 FC E4  #*!.........< ..
0001C150: E5 B2 9E C5 CA BD 74 28  BE 79 42 06 E6 A5 BC E6  ......t(.yB.....
0001C160: A2 AF 50 01 12 23 2A 21  08 B5 DA 01 10 C1 F1 02  ..P..#*!........
0001C170: 18 3C 20 EB BC AD E8 C3  88 FB EA 60 28 BE 79 42  .< ........`(.yB
0001C180: 06 E6 A5 BC E6 A2 AF 50  01 12 23 2A 21 08 E1 AC  .......P..#*!...
0001C190: 01 10 E2 F9 02 18 3C 20  F1 DE B5 A4 9C 84 B3 99  ......< ........
0001C1A0: 75 28 BE 79 42 06 E6 A5  BC E6 A2 AF 50 01 12 23  u(.yB.......P..#
0001C1B0: 2A 21 08 E1 79 10 D4 D3  02 18 3C 20 D9 A8 BB DD  *!..y.....< ....
0001C1C0: BC 91 B3 9A 81 01 28 BE  79 42 06 E6 A5 BC E6 A2  ......(.yB......

0001F750: F8 01 12 30 84 F3 06 F0  BC 0A E9 02 C5 10 A0 05  ...0............
0001F760: 71 E1 01 A5 09 84 08 AD  01 89 01 A1 07 D0 23 A1  q.............#.
0001F770: 06 88 01 84 06 8C 02 F0  0A 8C 02 F0 0A 58 EC 04  .............X..
0001F780: 9D 31 F4 08 12 1D 12 1B  08 F8 01 12 16 DC F3 06  .1..............
0001F790: 98 A2 0A 89 01 A1 07 84  08 AD 01 88 01 A0 07 85  ................
0001F7A0: 08 AC 01 12 F1 02 08 FA  A0 01 12 1D 12 1B 08 F8  ................
0001F7B0: 01 12 16 B8 C8 06 D0 BF  0A B5 01 A1 07 E4 09 E5  ................
0001F7C0: 01 B4 01 A0 07 E5 09 E4  01 12 1D 12 1B 08 F8 01  ................
0001F7D0: 12 16 84 C7 06 B0 B8 0A  B5 01 A1 07 E4 09 E5 01  ................
0001F7E0: B4 01 A0 07 E5 09 E4 01  12 1D 12 1B 08 F8 01 12  ................
0001F7F0: 16 D0 C5 06 90 B1 0A B5  01 A1 07 E4 09 E5 01 B4  ................
0001F800: 01 A0 07 E5 09 E4 01 12  1D 12 1B 08 F8 01 12 16  ................
0001F810: BC A1 07 A0 A4 0A 8D 02  F1 0A A0 0D D5 02 8C 02  ................

In [73]:
out_f = None
b = m1.get_block_xy((13531904, 3647232), 19)
bl.Clear()
bl.ParseFromString(b)
bl

b'\xd3\x10\xff\xf8' 21829 37670


layer {
  layerId: 7
  body {
    tag: 1604
    b2 {
      bg {
        data: "\001p\002\001_\026\027,;\002\005.W,\007,,\007$\305\276"
        mask: "\020\000\000\000\000\000"
      }
    }
  }
  body {
    tag: 1601
    b2 {
      bg {
        data: ":\002D\025\t9=\240\001C>\023\006"
        mask: "@\000\000\000"
      }
    }
    b2 {
      bg {
        data: "\001\020\002\001?\001NLD@5@1@/2"
        mask: "\021\000\000\000"
      }
    }
  }
  body {
    tag: 1549
    b2 {
      bg {
        data: "\340\262\003\r\005\225\237AA\0011d_\212\211\010\001\n\002\246\266\2450\001\r\014\r\002\013\002"
        mask: "\020\000\000\000\000\020\000\000"
      }
    }
    b2 {
      bg {
        data: "T\003b\001\275\257\240\265Z\0016.80\001F\001\001%&IR\021\0101>"
        mask: "P\000\000\000\000\000\000"
      }
    }
    b2 {
      bg {
        data: "\300\002*\003\241k\035\027/+y\207jg\016\nJej{nn+2\n\020\004\026\004\014* 2AHD\010\016\002\022\005\024\207\3129%\'<:&!8"
        mask: "P\000\000

In [74]:
out_f = None
b = m1.get_block_pos(444831 )
bl.Clear()
bl.ParseFromString(b)
bl
# f1(b,(100000,80000))

layer {
  layerId: 7
  body {
    tag: 2513
    b2 {
      bg {
        data: "\000\004\350\001\365\026\t\001\014\033\001\r\353=\001\035\244\001.\\\002#\0016"
        mask: "PD\000D\000"
      }
    }
    b2 {
      bg {
        data: "\000\004\350\001\365\026\t\001\014\033\001\r\353=\001\035\244\001.\\\002#\0016"
        mask: "PD\000D\000"
      }
    }
  }
  attr_3: 2000
  attr_4: 0
}
layer {
  layerId: 15
  body {
    tag: 61229
    b2 {
      bg {
        data: "\274\374\000 \003a\t\001\370\241\001\001\370\003\005\033\330\035\006\320\003)\035\000\004\r\004<\tE\001\020\335h\002\031`\t\t\334q\002Y\005\274\301\003\264\034U\004,\035\371D\001\001"
        mask: "d%\025UQTU "
      }
    }
    b2 {
      bg {
        data: "\000 \003x\\\341\247i0\234\002\005\tD\245\254/\001\300\t"
        mask: "\225U\020"
      }
    }
    b2 {
      bg {
        data: "\3509\002p\363\341\010\371\002\360?m\276\230\305\0025Y\002\001\001\361\001\315\001\205\001\231\002\035\001\271C\211\024\321\016u\004\2

In [46]:
if abdsfo:
    pass

NameError: name 'abdsfo' is not defined

In [271]:
n = 0
p = 512
while p < 287232:
    v = struct.unpack('i', m1.idx_part[p:p+4])[0]
    if v > 287232:
        n += 1
    p += 4
n

48418

In [301]:
# hexdump.hexdump(m1.idx_part)

In [133]:
hex(317444 - 0x4000 - 0x1000 - 0x400 - 0x4), hex(58100 - 0x10*5)

('0x48400', '0xe2a4')

In [15]:
hex(309504), hex(0x4d804 +0x100 + 0x100 ), 0x4d804

('0x4da04', '0x4da04', 317444)

In [69]:
xy_2_idx((13490348 + 13491991)//2, (3585779 + 3584323)//2, 19)
idx_2_xy((1715, 43, 29, 21), 19)
xy = (13490348 + 13491991)//2, (3585779 + 3584323)//2
hexdump.hexdump(m1.get_block_xy(xy, 19))
print('-------------------------')
hexdump.hexdump(decode_2(m1.get_block_xy(xy, 12)))
# hexdump.hexdump(guess_decode(m1.get_glock_xy(xy, 19)))

decode_block_header(m1.get_block_xy(xy, 12))

00000000: F8 9E FF F8 58 91 FF F8  01 05 00 00 50 9D FF F8  ....X.......P...
-------------------------
00000000: 00 00 00 00 A0 0F 00 00  2C D5 01 00 56 96 00 00  ........,...V...


(b'\xcd\xce\xfd\xf8', 38486, 120108)

In [80]:
m1.get_block_xy(xy, 19)

b'\xf8\x9e\xff\xf8X\x91\xff\xf8\x01\x05\x00\x00P\x9d\xff\xf8'

In [302]:
# hexdump.hexdump(m1.get_block_xy(xy, 19))
# m1.get_block_xy(xy, 19)

In [303]:
# hexdump.hexdump(m1.get_block_xy(xy, 19))

In [107]:
cc = m1.get_block_xy(xy, 12)
import zlib


dd = dz.decompress(cc)
# len(dd)

b'\xcd\xce\xfd\xf8' 38486 120108


error: Error -3 while decompressing data: incorrect data check

In [101]:
open('/tmp/z.bin', 'wb').write(cc)

38486

In [104]:
dz = zlib.decompressobj()
u = dz.decompress(cc)
len(u)

120118

In [122]:
# hexdump.hexdump(cc)

In [53]:
b = m1.get_block_xy((13505679, 3636291), 19)

b'\xae\x8f\xfd\xf8' 73216 136267


In [54]:
open('/tmp/indoor.bin', 'wb').write(b)

136267

In [79]:
bbc = bl.ParseFromString(b)

In [149]:
g_tmp['5087719067573934800'][3]['floors'][2]['name']

'M'

In [124]:
len(g_tmp)

3

In [90]:
g_tmp = {}

bbb = []
out_f = None
f_indoor(bl, (0, 0))

-------------------------------------------------
floor {
  name: "B2"
  idx_num: -3
  b1 {
    b2 {
      some_id: 3
      b3 {
        some_id_b3: 18
        b4 {
          poi {
            x: 7613
            y: 4662
            attr_3: 192
            attr_4: 5087719067573934800
            attr_5: 20000
            name: "\350\231\271\346\241\245\345\244\251\345\234\260\\\350\264\255\347\211\251\344\270\255\345\277\203"
            attr_10: 1
          }
        }
      }
      b3 {
        some_id_b3: 57
        b4 {
          poi {
            x: 12202
            y: 269
            attr_3: 252
            attr_4: 4371708599591697449
            attr_5: 19500
            name: "\345\201\234\350\275\246\345\234\272\345\207\272\345\205\245\345\217\243"
            attr_10: 1
          }
        }
      }
      b3 {
        some_id_b3: 158
        b4 {
          poi {
            x: 5562
            y: 11995
            attr_3: 60
            attr_4: 6194376336043092866
          